In [1]:
# -*- coding: utf-8 -*-
import os
import datetime as dt
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

import tushare as ts
import statsmodels.api as sm

############################################
# 0. 参数与准备
############################################
TUSHARE_TOKEN = "cb8d6c9ce12cf06aa54603af6a26ab4c660d1cfb2e20b0609872e051"
ts.set_token(TUSHARE_TOKEN)
pro = ts.pro_api(TUSHARE_TOKEN)

# 时间参数：近五年，按月
TODAY = dt.date.today()
END_DATE = TODAY.strftime("%Y%m%d")
START_DATE = (TODAY.replace(day=1) - pd.DateOffset(years=5)).date().strftime("%Y%m%d")

def month_end(d):
    return (pd.to_datetime(d) + pd.offsets.MonthEnd(0))

# 设定本地保存/读取目录（使用你指定的绝对路径）
PANEL_DIR = "/Users/xiaoquanliu/Desktop/RAF628/Week7/"
PANEL_BASE_NAME = "sz50_ff3_panel"

# ==========================================
# 工具函数：保存与加载面板
# ==========================================
def save_panel(panel: pd.DataFrame, out_dir: str = PANEL_DIR, base_name: str = PANEL_BASE_NAME):
    os.makedirs(out_dir, exist_ok=True)
    pq_path = os.path.join(out_dir, f"{base_name}.parquet")
    csv_path = os.path.join(out_dir, f"{base_name}.csv")
    df = panel.copy()
    # 确保 trade_month 为 datetime64
    df["trade_month"] = pd.to_datetime(df["trade_month"])
    # 保存 Parquet
    try:
        df.to_parquet(pq_path, index=False)
        print(f"[INFO] 面板已保存：{pq_path}  (推荐使用此文件)")
    except Exception as e:
        print(f"[WARN] 保存 Parquet 失败：{e}")
    # 保存 CSV 兜底
    try:
        df.to_csv(csv_path, index=False, encoding="utf-8")
        print(f"[INFO] 面板已保存：{csv_path}")
    except Exception as e:
        print(f"[WARN] 保存 CSV 失败：{e}")

def load_panel(in_dir: str = PANEL_DIR, base_name: str = PANEL_BASE_NAME) -> pd.DataFrame:
    pq_path = os.path.join(in_dir, f"{base_name}.parquet")
    csv_path = os.path.join(in_dir, f"{base_name}.csv")
    if os.path.exists(pq_path):
        df = pd.read_parquet(pq_path)
        print(f("[INFO] 已从本地加载面板（Parquet）：{pq_path}"))
    elif os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        print(f("[INFO] 已从本地加载面板（CSV）：{csv_path}"))
    else:
        raise FileNotFoundError(f"未找到本地面板文件：{pq_path} 或 {csv_path}")
    # 类型修正
    df["trade_month"] = pd.to_datetime(df["trade_month"])
    # 关键数值列确保为数值
    for col in ["ret","RF","MKT_RF","SMB","HML","excess_ret","MKT_ret"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

############################################
# 1. 上证50成分 + 月度收益
############################################
def get_sz50_constituents():
    # 用 index_weight 取成分，选最近一期
    weight = pro.index_weight(index_code="000016.SH", start_date=START_DATE, end_date=END_DATE)
    if weight is None or weight.empty:
        # 尝试只取 end_date 当月
        weight = pro.index_weight(index_code="000016.SH", trade_date=END_DATE)
    if weight is None or weight.empty:
        raise ValueError("TuShare index_weight 未取到上证50成分，请检查权限或日期区间。")
    # 取最新交易月
    weight["trade_date"] = pd.to_datetime(weight["trade_date"])
    latest_month = weight["trade_date"].max()
    latest_cons = weight.loc[weight["trade_date"] == latest_month, "con_code"].dropna().unique().tolist()
    return sorted(latest_cons)

def get_monthly_returns_for_stocks(ts_codes, start_date, end_date):
    """
    使用前复权因子构造月度收益（adj_close），返回 [trade_month, ts_code, ret]
    """
    out = []
    for code in tqdm(ts_codes, desc="SZ50 daily kline"):
        try:
            # 日线
            df = pro.daily(ts_code=code, start_date=start_date, end_date=end_date)
            if df is None or df.empty:
                continue
            df["trade_date"] = pd.to_datetime(df["trade_date"])
            df.sort_values("trade_date", inplace=True)

            # 复权因子
            adj = pro.adj_factor(ts_code=code, start_date=start_date, end_date=end_date)
            if adj is None or adj.empty:
                continue
            adj["trade_date"] = pd.to_datetime(adj["trade_date"])

            m = df.merge(adj[["trade_date", "adj_factor"]], on="trade_date", how="left")
            m["adj_close"] = m["close"] * m["adj_factor"]
            m["trade_month"] = month_end(m["trade_date"])
            month_px = m.groupby("trade_month", as_index=False)["adj_close"].last()
            month_px["ret"] = month_px["adj_close"].pct_change()
            tmp = month_px.dropna().copy()
            tmp["ts_code"] = code
            out.append(tmp[["trade_month", "ts_code", "ret"]])
        except Exception as e:
            print(f"[WARN] {code} daily/adjs fail: {e}")
            continue
    if len(out) == 0:
        return pd.DataFrame(columns=["trade_month", "ts_code", "ret"])
    ret = pd.concat(out, ignore_index=True)
    # 限定区间
    ret = ret[(ret["trade_month"] >= month_end(START_DATE)) & (ret["trade_month"] <= month_end(END_DATE))]
    return ret

############################################
# 2. 构造 FF3 因子
############################################
def get_universe_mv_pb(start_date, end_date):
    """
    用 daily_basic 获取全市场的月末 total_mv 与 pb，构造 B/M 近似：BM = 1/PB
    返回列：['trade_month','ts_code','mv','pb','bm_proxy']
    """
    basic = pro.stock_basic(exchange='', list_status='L', fields='ts_code,list_date')
    codes = basic["ts_code"].tolist()
    out = []
    for code in tqdm(codes, desc="Universe mv/pb (daily_basic)"):
        try:
            db = pro.daily_basic(ts_code=code, start_date=start_date, end_date=end_date,
                                 fields="ts_code,trade_date,total_mv,pb")
            if db is None or db.empty:
                continue
            db["trade_date"] = pd.to_datetime(db["trade_date"])
            db["trade_month"] = month_end(db["trade_date"])
            month_last = db.sort_values("trade_date").groupby("trade_month", as_index=False).last()
            month_last.rename(columns={"total_mv": "mv"}, inplace=True)
            out.append(month_last[["trade_month", "ts_code", "mv", "pb"]])
        except:
            continue
    if len(out) == 0:
        return pd.DataFrame(columns=["trade_month","ts_code","mv","pb","bm_proxy"])
    panel = pd.concat(out, ignore_index=True)
    panel["bm_proxy"] = 1.0 / panel["pb"]
    panel.replace([np.inf, -np.inf], np.nan, inplace=True)
    panel = panel.dropna(subset=["mv"])  # 至少有市值
    return panel

def get_universe_monthly_returns(start_date, end_date, codes=None):
    """
    获取全市场（或给定集合）月度收益，使用未复权收盘价的月末 last 计算 pct_change。
    返回列：['trade_month','ts_code','ret']
    """
    if codes is None:
        basic = pro.stock_basic(exchange='', list_status='L', fields='ts_code')
        codes = basic["ts_code"].tolist()
    out = []
    for code in tqdm(codes, desc="Universe monthly returns"):
        try:
            df = pro.daily(ts_code=code, start_date=start_date, end_date=end_date)
            if df is None or df.empty:
                continue
            df["trade_date"] = pd.to_datetime(df["trade_date"])
            df.sort_values("trade_date", inplace=True)
            df["trade_month"] = month_end(df["trade_date"])
            month_px = df.groupby("trade_month", as_index=False)["close"].last()
            month_px["ret"] = month_px["close"].pct_change()
            tmp = month_px.dropna().copy()
            tmp["ts_code"] = code
            out.append(tmp[["trade_month", "ts_code", "ret"]])
        except:
            continue
    if len(out) == 0:
        return pd.DataFrame(columns=["trade_month","ts_code","ret"])
    ret = pd.concat(out, ignore_index=True)
    ret = ret[(ret["trade_month"] >= month_end(START_DATE)) & (ret["trade_month"] <= month_end(END_DATE))]
    return ret

def compute_ff3_factors(universe_ret, mv_pb_panel):
    """
    输入：
    - universe_ret: ['trade_month','ts_code','ret']
    - mv_pb_panel:  ['trade_month','ts_code','mv','pb','bm_proxy']
    输出：['trade_month','RF','MKT_RF','SMB','HML','MKT_ret']
    """
    # 合并市值/估算BM
    df = universe_ret.merge(mv_pb_panel, on=["trade_month","ts_code"], how="left")
    df = df.dropna(subset=["mv"])  # 市值缺失剔除

    # 计算 MKT（市值加权）
    def vwret(g):
        g2 = g.dropna(subset=["ret","mv"])
        if g2.empty:
            return np.nan
        w = g2["mv"].astype(float).values
        w = w / np.nansum(w)
        return np.nansum(w * g2["ret"].values)
    mkt = df.groupby("trade_month").apply(vwret).rename("MKT_ret").reset_index()

    # 无风险利率 RF：用 SHIBOR 1M 转月化
    shibor = pro.shibor(start_date=START_DATE, end_date=END_DATE)  # 列：date, 1m, ...
    shibor["date"] = pd.to_datetime(shibor["date"])
    shibor["trade_month"] = month_end(shibor["date"])
    shibor_m = shibor.groupby("trade_month", as_index=False)["1m"].last()
    shibor_m["RF"] = (1.0 + shibor_m["1m"] / 100.0) ** (1/12.0) - 1.0
    rf = shibor_m[["trade_month", "RF"]]

    # SMB/HML：每月做 2x3 切分并取等权收益
    def smb_hml_by_month(g):
        g = g.dropna(subset=["ret","mv"])
        if g.shape[0] < 100:
            return pd.Series({"SMB": np.nan, "HML": np.nan})
        # Size: 中位数分割
        mv_median = g["mv"].median()
        g["size"] = np.where(g["mv"] <= mv_median, "S", "B")
        # BM: 30/70 分位
        q = g["bm_proxy"].quantile([0.3, 0.7])
        lo, hi = q.iloc[0], q.iloc[1]
        def tag_bm(x):
            if pd.isna(x):
                return np.nan
            if x <= lo:
                return "L"
            elif x <= hi:
                return "M"
            else:
                return "H"
        g["bm"] = g["bm_proxy"].apply(tag_bm)
        g = g.dropna(subset=["bm"])
        def ew(df_):
            return df_["ret"].mean() if df_.shape[0] > 0 else np.nan

        SL = ew(g[(g["size"]=="S") & (g["bm"]=="L")])
        SM = ew(g[(g["size"]=="S") & (g["bm"]=="M")])
        SH = ew(g[(g["size"]=="S") & (g["bm"]=="H")])
        BL = ew(g[(g["size"]=="B") & (g["bm"]=="L")])
        BM = ew(g[(g["size"]=="B") & (g["bm"]=="M")])
        BH = ew(g[(g["size"]=="B") & (g["bm"]=="H")])

        SMB = np.nanmean([SL, SM, SH]) - np.nanmean([BL, BM, BH])
        H   = np.nanmean([SH, BH])
        L   = np.nanmean([SL, BL])
        HML = H - L
        return pd.Series({"SMB": SMB, "HML": HML})

    smb_hml = df.groupby("trade_month").apply(smb_hml_by_month).reset_index()

    ff3 = mkt.merge(rf, on="trade_month", how="inner").merge(smb_hml, on="trade_month", how="inner")
    ff3["MKT_RF"] = ff3["MKT_ret"] - ff3["RF"]
    ff3 = ff3.sort_values("trade_month").dropna(subset=["MKT_RF","SMB","HML"])
    return ff3[["trade_month","RF","MKT_RF","SMB","HML","MKT_ret"]]

############################################
# 3. 合并三因子与上证50个股收益，构造面板
############################################
def build_panel(ff3, sz50_ret):
    panel = sz50_ret.merge(ff3, on="trade_month", how="inner")
    panel["excess_ret"] = panel["ret"] - panel["RF"]
    panel = panel.replace([np.inf, -np.inf], np.nan).dropna(subset=["excess_ret","MKT_RF","SMB","HML"])
    return panel

############################################
# 4. Fama–MacBeth 两步
############################################
def fama_macbeth(panel, use_hac: bool = False, hac_lags: int = 3):
    # Step 1: 每只股票时间序列回归，估计 beta
    rows = []
    for code, g in panel.groupby("ts_code"):
        g = g.sort_values("trade_month")
        if g.shape[0] < 24:  # 至少两年
            continue
        y = g["excess_ret"].values
        X = g[["MKT_RF","SMB","HML"]].values
        X = sm.add_constant(X)
        model = sm.OLS(y, X).fit()
        rows.append({
            "ts_code": code,
            "alpha": model.params[0], "beta_MKT": model.params[1], "beta_SMB": model.params[2], "beta_HML": model.params[3],
            "t_alpha": model.tvalues[0], "t_beta_MKT": model.tvalues[1], "t_beta_SMB": model.tvalues[2], "t_beta_HML": model.tvalues[3],
            "n_obs": int(model.nobs), "r2": model.rsquared
        })
    step1 = pd.DataFrame(rows)
    if step1.empty:
        raise ValueError("Step1 无有效回归结果，可能是样本过短或数据缺失。")

    betas = step1[["ts_code","beta_MKT","beta_SMB","beta_HML"]]

    # Step 2: 每月横截面回归：excess_ret_it on betas
    merged = panel.merge(betas, on="ts_code", how="inner")
    lam_rows = []
    for t, g in merged.groupby("trade_month"):
        if g.shape[0] < 10:
            continue
        y = g["excess_ret"].values
        X = g[["beta_MKT","beta_SMB","beta_HML"]].values
        X = sm.add_constant(X)
        model = sm.OLS(y, X).fit()
        lam_rows.append({
            "trade_month": t,
            "lambda_0": model.params[0],
            "lambda_MKT": model.params[1],
            "lambda_SMB": model.params[2],
            "lambda_HML": model.params[3],
            "n_cs": int(model.nobs)
        })
    lambda_df = pd.DataFrame(lam_rows).sort_values("trade_month")

    # 对 lambda 时间序列做均值与 t 值
    def summarize_simple(series):
        s = series.dropna()
        n = len(s)
        mean = s.mean()
        std = s.std(ddof=1)
        tval = mean / (std / np.sqrt(n)) if (std > 0 and n > 1) else np.nan
        return mean, std, n, tval

    def summarize_hac(series, lags: int = 3):
        s = series.dropna().values
        n = len(s)
        if n <= 1:
            return np.nan, np.nan, n, np.nan
        # 回归: s_t = mu + e_t，取常数项的HAC标准误
        y = s
        X = np.ones((n, 1))
        model = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": lags})
        mean = model.params[0]
        se = model.bse[0]
        tval = mean / se if se > 0 else np.nan
        return mean, se, n, tval

    stats_rows = []
    if use_hac:
        for col in ["lambda_0","lambda_MKT","lambda_SMB","lambda_HML"]:
            mean, se, n, tval = summarize_hac(lambda_df[col], lags=hac_lags)
            stats_rows.append({"param": col, "mean": mean, "se_or_std": se, "n": n, "t": tval})
    else:
        for col in ["lambda_0","lambda_MKT","lambda_SMB","lambda_HML"]:
            mean, std, n, tval = summarize_simple(lambda_df[col])
            stats_rows.append({"param": col, "mean": mean, "se_or_std": std, "n": n, "t": tval})
    summary = pd.DataFrame(stats_rows)

    return step1, lambda_df, summary

############################################
# 主流程
############################################
def main(use_local_panel: bool = True,
         panel_dir: str = PANEL_DIR,
         panel_base_name: str = PANEL_BASE_NAME,
         use_hac: bool = False,
         hac_lags: int = 3):
    if not use_local_panel:
        print("1) 拉取上证50成分")
        sz50 = get_sz50_constituents()
        print(f"上证50成分股数量: {len(sz50)}")

        print("2) 计算上证50个股月度收益")
        sz50_ret = get_monthly_returns_for_stocks(sz50, START_DATE, END_DATE)
        print(f"上证50个股月收益条目: {sz50_ret.shape[0]}")

        print("3) 构造全市场 MV/PB 面板")
        mv_pb = get_universe_mv_pb(START_DATE, END_DATE)
        print(f"全市场 MV/PB 面板条目: {mv_pb.shape[0]}")

        print("4) 获取全市场月度收益并计算 FF3 因子")
        univ_ret = get_universe_monthly_returns(START_DATE, END_DATE)
        ff3 = compute_ff3_factors(univ_ret, mv_pb)
        print(f"FF3 因子月份数: {ff3.shape[0]}")

        print("5) 合并面板")
        panel = build_panel(ff3, sz50_ret)
        print(f"面板数据条目: {panel.shape[0]}")

        # 保存到本地
        save_panel(panel, out_dir=panel_dir, base_name=panel_base_name)

    else:
        print("跳过数据抓取，直接从本地加载面板...")
        panel = load_panel(in_dir=panel_dir, base_name=panel_base_name)
        print(f"本地面板数据条目: {panel.shape[0]}")

    print("6) Fama–MacBeth 两步回归")
    step1, lambda_df, summary = fama_macbeth(panel, use_hac=use_hac, hac_lags=hac_lags)

    print("\n第一步：时间序列回归（前5行）")
    print(step1.head())

    print("\n第二步：横截面回归（前5行）")
    print(lambda_df.head())

    print("\n第二步：因子价格统计（均值/SE或STD、样本量、t值）")
    print(summary)

if __name__ == "__main__":
    # 首次运行：抓取→计算→保存→回归
    main(use_local_panel=False, use_hac=False)

 

1) 拉取上证50成分
上证50成分股数量: 50
2) 计算上证50个股月度收益


SZ50 daily kline: 100%|██████████| 50/50 [00:46<00:00,  1.08it/s]


上证50个股月收益条目: 2953
3) 构造全市场 MV/PB 面板


Universe mv/pb (daily_basic): 100%|██████████| 5439/5439 [35:39<00:00,  2.54it/s]  


全市场 MV/PB 面板条目: 298373
4) 获取全市场月度收益并计算 FF3 因子


Universe monthly returns: 100%|██████████| 5439/5439 [42:57<00:00,  2.11it/s]  


FF3 因子月份数: 60
5) 合并面板
面板数据条目: 2953
[INFO] 面板已保存：/Users/xiaoquanliu/Desktop/RAF628/Week7/sz50_ff3_panel.parquet  (推荐使用此文件)
[INFO] 面板已保存：/Users/xiaoquanliu/Desktop/RAF628/Week7/sz50_ff3_panel.csv
6) Fama–MacBeth 两步回归

第一步：时间序列回归（前5行）
     ts_code     alpha  beta_MKT  beta_SMB  beta_HML   t_alpha  t_beta_MKT  \
0  600028.SH  0.028151  0.431267 -0.030675  0.750231  3.384257    2.756930   
1  600030.SH  0.003876  1.496035  0.167902  0.520344  0.409746    8.409382   
2  600031.SH -0.004297  1.332855 -0.355769  0.439649 -0.417381    6.883603   
3  600036.SH  0.014761  1.096164 -0.225102  0.794373  1.414802    5.586590   
4  600048.SH  0.014346  1.395142  0.490073  1.046633  1.041179    5.384007   

   t_beta_SMB  t_beta_HML  n_obs        r2  
0   -0.146550    4.811411     60  0.313253  
1    0.705355    2.934338     60  0.562076  
2   -1.373187    2.277911     60  0.498313  
3   -0.857393    4.061568     60  0.424766  
4    1.413439    4.052094     60  0.381545  

第二步：横截面回归（前5行）
  trade_month

In [2]:
# -*- coding: utf-8 -*-
import os
import datetime as dt
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

import tushare as ts
import statsmodels.api as sm

############################################
# 0. 参数与准备
############################################
# 你的 TuShare Token（如需安全请改为环境变量方式）
TUSHARE_TOKEN = "cb8d6c9ce12cf06aa54603af6a26ab4c660d1cfb2e20b0609872e051"
ts.set_token(TUSHARE_TOKEN)
pro = ts.pro_api(TUSHARE_TOKEN)

# 时间参数：近五年，按月
TODAY = dt.date.today()
END_DATE = TODAY.strftime("%Y%m%d")
START_DATE = (TODAY.replace(day=1) - pd.DateOffset(years=5)).date().strftime("%Y%m%d")

def month_end(d):
    return (pd.to_datetime(d) + pd.offsets.MonthEnd(0))

# 设定本地保存/读取目录（使用你指定的绝对路径）
PANEL_DIR = "/Users/xiaoquanliu/Desktop/RAF628/Week7/"
PANEL_BASE_NAME = "sz50_ff3_panel"

# ==========================================
# 工具函数：保存与加载面板
# ==========================================
def save_panel(panel: pd.DataFrame, out_dir: str = PANEL_DIR, base_name: str = PANEL_BASE_NAME):
    os.makedirs(out_dir, exist_ok=True)
    pq_path = os.path.join(out_dir, f"{base_name}.parquet")
    csv_path = os.path.join(out_dir, f"{base_name}.csv")
    df = panel.copy()
    # 确保 trade_month 为 datetime64
    df["trade_month"] = pd.to_datetime(df["trade_month"])
    # 保存 Parquet
    try:
        df.to_parquet(pq_path, index=False)
        print(f"[INFO] 面板已保存：{pq_path}  (推荐使用此文件)")
    except Exception as e:
        print(f"[WARN] 保存 Parquet 失败：{e}")
    # 保存 CSV 兜底
    try:
        df.to_csv(csv_path, index=False, encoding="utf-8")
        print(f"[INFO] 面板已保存：{csv_path}")
    except Exception as e:
        print(f"[WARN] 保存 CSV 失败：{e}")

def load_panel(in_dir: str = PANEL_DIR, base_name: str = PANEL_BASE_NAME) -> pd.DataFrame:
    pq_path = os.path.join(in_dir, f"{base_name}.parquet")
    csv_path = os.path.join(in_dir, f"{base_name}.csv")
    if os.path.exists(pq_path):
        df = pd.read_parquet(pq_path)
        print(f("[INFO] 已从本地加载面板（Parquet）：{pq_path}"))
    elif os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        print(f("[INFO] 已从本地加载面板（CSV）：{csv_path}"))
    else:
        raise FileNotFoundError(f"未找到本地面板文件：{pq_path} 或 {csv_path}")
    # 类型修正
    df["trade_month"] = pd.to_datetime(df["trade_month"])
    # 关键数值列确保为数值
    for col in ["ret","RF","MKT_RF","SMB","HML","excess_ret","MKT_ret"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

############################################
# 1. 上证50成分 + 月度收益
############################################
def get_sz50_constituents():
    # 用 index_weight 取成分，选最近一期
    weight = pro.index_weight(index_code="000016.SH", start_date=START_DATE, end_date=END_DATE)
    if weight is None or weight.empty:
        # 尝试只取 end_date 当月
        weight = pro.index_weight(index_code="000016.SH", trade_date=END_DATE)
    if weight is None or weight.empty:
        raise ValueError("TuShare index_weight 未取到上证50成分，请检查权限或日期区间。")
    # 取最新交易月
    weight["trade_date"] = pd.to_datetime(weight["trade_date"])
    latest_month = weight["trade_date"].max()
    latest_cons = weight.loc[weight["trade_date"] == latest_month, "con_code"].dropna().unique().tolist()
    return sorted(latest_cons)

def get_monthly_returns_for_stocks(ts_codes, start_date, end_date):
    """
    使用前复权因子构造月度收益（adj_close），返回 [trade_month, ts_code, ret]
    """
    out = []
    for code in tqdm(ts_codes, desc="SZ50 daily kline"):
        try:
            # 日线
            df = pro.daily(ts_code=code, start_date=start_date, end_date=end_date)
            if df is None or df.empty:
                continue
            df["trade_date"] = pd.to_datetime(df["trade_date"])
            df.sort_values("trade_date", inplace=True)

            # 复权因子
            adj = pro.adj_factor(ts_code=code, start_date=start_date, end_date=end_date)
            if adj is None or adj.empty:
                continue
            adj["trade_date"] = pd.to_datetime(adj["trade_date"])

            m = df.merge(adj[["trade_date", "adj_factor"]], on="trade_date", how="left")
            m["adj_close"] = m["close"] * m["adj_factor"]
            m["trade_month"] = month_end(m["trade_date"])
            month_px = m.groupby("trade_month", as_index=False)["adj_close"].last()
            month_px["ret"] = month_px["adj_close"].pct_change()
            tmp = month_px.dropna().copy()
            tmp["ts_code"] = code
            out.append(tmp[["trade_month", "ts_code", "ret"]])
        except Exception as e:
            print(f"[WARN] {code} daily/adjs fail: {e}")
            continue
    if len(out) == 0:
        return pd.DataFrame(columns=["trade_month", "ts_code", "ret"])
    ret = pd.concat(out, ignore_index=True)
    # 限定区间
    ret = ret[(ret["trade_month"] >= month_end(START_DATE)) & (ret["trade_month"] <= month_end(END_DATE))]
    return ret

############################################
# 2. 构造 FF3 因子
############################################
def get_universe_mv_pb(start_date, end_date):
    """
    用 daily_basic 获取全市场的月末 total_mv 与 pb，构造 B/M 近似：BM = 1/PB
    返回列：['trade_month','ts_code','mv','pb','bm_proxy']
    """
    basic = pro.stock_basic(exchange='', list_status='L', fields='ts_code,list_date')
    codes = basic["ts_code"].tolist()
    out = []
    for code in tqdm(codes, desc="Universe mv/pb (daily_basic)"):
        try:
            db = pro.daily_basic(ts_code=code, start_date=start_date, end_date=end_date,
                                 fields="ts_code,trade_date,total_mv,pb")
            if db is None or db.empty:
                continue
            db["trade_date"] = pd.to_datetime(db["trade_date"])
            db["trade_month"] = month_end(db["trade_date"])
            month_last = db.sort_values("trade_date").groupby("trade_month", as_index=False).last()
            month_last.rename(columns={"total_mv": "mv"}, inplace=True)
            out.append(month_last[["trade_month", "ts_code", "mv", "pb"]])
        except:
            continue
    if len(out) == 0:
        return pd.DataFrame(columns=["trade_month","ts_code","mv","pb","bm_proxy"])
    panel = pd.concat(out, ignore_index=True)
    panel["bm_proxy"] = 1.0 / panel["pb"]
    panel.replace([np.inf, -np.inf], np.nan, inplace=True)
    panel = panel.dropna(subset=["mv"])  # 至少有市值
    return panel

def get_universe_monthly_returns(start_date, end_date, codes=None):
    """
    获取全市场（或给定集合）月度收益，使用未复权收盘价的月末 last 计算 pct_change。
    返回列：['trade_month','ts_code','ret']
    """
    if codes is None:
        basic = pro.stock_basic(exchange='', list_status='L', fields='ts_code')
        codes = basic["ts_code"].tolist()
    out = []
    for code in tqdm(codes, desc="Universe monthly returns"):
        try:
            df = pro.daily(ts_code=code, start_date=start_date, end_date=end_date)
            if df is None or df.empty:
                continue
            df["trade_date"] = pd.to_datetime(df["trade_date"])
            df.sort_values("trade_date", inplace=True)
            df["trade_month"] = month_end(df["trade_date"])
            month_px = df.groupby("trade_month", as_index=False)["close"].last()
            month_px["ret"] = month_px["close"].pct_change()
            tmp = month_px.dropna().copy()
            tmp["ts_code"] = code
            out.append(tmp[["trade_month", "ts_code", "ret"]])
        except:
            continue
    if len(out) == 0:
        return pd.DataFrame(columns=["trade_month","ts_code","ret"])
    ret = pd.concat(out, ignore_index=True)
    ret = ret[(ret["trade_month"] >= month_end(START_DATE)) & (ret["trade_month"] <= month_end(END_DATE))]
    return ret

def compute_ff3_factors(universe_ret, mv_pb_panel):
    """
    输入：
    - universe_ret: ['trade_month','ts_code','ret']
    - mv_pb_panel:  ['trade_month','ts_code','mv','pb','bm_proxy']
    输出：['trade_month','RF','MKT_RF','SMB','HML','MKT_ret']
    """
    # 合并市值/估算BM
    df = universe_ret.merge(mv_pb_panel, on=["trade_month","ts_code"], how="left")
    df = df.dropna(subset=["mv"])  # 市值缺失剔除

    # 计算 MKT（市值加权）
    def vwret(g):
        g2 = g.dropna(subset=["ret","mv"])
        if g2.empty:
            return np.nan
        w = g2["mv"].astype(float).values
        w = w / np.nansum(w)
        return np.nansum(w * g2["ret"].values)
    mkt = df.groupby("trade_month").apply(vwret).rename("MKT_ret").reset_index()

    # 无风险利率 RF：用 SHIBOR 1M 转月化
    shibor = pro.shibor(start_date=START_DATE, end_date=END_DATE)  # 列：date, 1m, ...
    shibor["date"] = pd.to_datetime(shibor["date"])
    shibor["trade_month"] = month_end(shibor["date"])
    shibor_m = shibor.groupby("trade_month", as_index=False)["1m"].last()
    shibor_m["RF"] = (1.0 + shibor_m["1m"] / 100.0) ** (1/12.0) - 1.0
    rf = shibor_m[["trade_month", "RF"]]

    # SMB/HML：每月做 2x3 切分并取等权收益
    def smb_hml_by_month(g):
        g = g.dropna(subset=["ret","mv"])
        if g.shape[0] < 100:
            return pd.Series({"SMB": np.nan, "HML": np.nan})
        # Size: 中位数分割
        mv_median = g["mv"].median()
        g["size"] = np.where(g["mv"] <= mv_median, "S", "B")
        # BM: 30/70 分位
        q = g["bm_proxy"].quantile([0.3, 0.7])
        lo, hi = q.iloc[0], q.iloc[1]
        def tag_bm(x):
            if pd.isna(x):
                return np.nan
            if x <= lo:
                return "L"
            elif x <= hi:
                return "M"
            else:
                return "H"
        g["bm"] = g["bm_proxy"].apply(tag_bm)
        g = g.dropna(subset=["bm"])
        def ew(df_):
            return df_["ret"].mean() if df_.shape[0] > 0 else np.nan

        SL = ew(g[(g["size"]=="S") & (g["bm"]=="L")])
        SM = ew(g[(g["size"]=="S") & (g["bm"]=="M")])
        SH = ew(g[(g["size"]=="S") & (g["bm"]=="H")])
        BL = ew(g[(g["size"]=="B") & (g["bm"]=="L")])
        BM = ew(g[(g["size"]=="B") & (g["bm"]=="M")])
        BH = ew(g[(g["size"]=="B") & (g["bm"]=="H")])

        SMB = np.nanmean([SL, SM, SH]) - np.nanmean([BL, BM, BH])
        H   = np.nanmean([SH, BH])
        L   = np.nanmean([SL, BL])
        HML = H - L
        return pd.Series({"SMB": SMB, "HML": HML})

    smb_hml = df.groupby("trade_month").apply(smb_hml_by_month).reset_index()

    ff3 = mkt.merge(rf, on="trade_month", how="inner").merge(smb_hml, on="trade_month", how="inner")
    ff3["MKT_RF"] = ff3["MKT_ret"] - ff3["RF"]
    ff3 = ff3.sort_values("trade_month").dropna(subset=["MKT_RF","SMB","HML"])
    return ff3[["trade_month","RF","MKT_RF","SMB","HML","MKT_ret"]]

############################################
# 3. 合并三因子与上证50个股收益，构造面板
############################################
def build_panel(ff3, sz50_ret):
    panel = sz50_ret.merge(ff3, on="trade_month", how="inner")
    panel["excess_ret"] = panel["ret"] - panel["RF"]
    panel = panel.replace([np.inf, -np.inf], np.nan).dropna(subset=["excess_ret","MKT_RF","SMB","HML"])
    return panel

############################################
# 4. Fama–MacBeth 两步
############################################
def fama_macbeth(panel, use_hac: bool = False, hac_lags: int = 3):
    # Step 1: 每只股票时间序列回归，估计 beta
    rows = []
    for code, g in panel.groupby("ts_code"):
        g = g.sort_values("trade_month")
        if g.shape[0] < 24:  # 至少两年
            continue
        y = g["excess_ret"].values
        X = g[["MKT_RF","SMB","HML"]].values
        X = sm.add_constant(X)
        model = sm.OLS(y, X).fit()
        rows.append({
            "ts_code": code,
            "alpha": model.params[0], "beta_MKT": model.params[1], "beta_SMB": model.params[2], "beta_HML": model.params[3],
            "t_alpha": model.tvalues[0], "t_beta_MKT": model.tvalues[1], "t_beta_SMB": model.tvalues[2], "t_beta_HML": model.tvalues[3],
            "n_obs": int(model.nobs), "r2": model.rsquared
        })
    step1 = pd.DataFrame(rows)
    if step1.empty:
        raise ValueError("Step1 无有效回归结果，可能是样本过短或数据缺失。")

    betas = step1[["ts_code","beta_MKT","beta_SMB","beta_HML"]]

    # Step 2: 每月横截面回归：excess_ret_it on betas
    merged = panel.merge(betas, on="ts_code", how="inner")
    lam_rows = []
    for t, g in merged.groupby("trade_month"):
        if g.shape[0] < 10:
            continue
        y = g["excess_ret"].values
        X = g[["beta_MKT","beta_SMB","beta_HML"]].values
        X = sm.add_constant(X)
        model = sm.OLS(y, X).fit()
        lam_rows.append({
            "trade_month": t,
            "lambda_0": model.params[0],
            "lambda_MKT": model.params[1],
            "lambda_SMB": model.params[2],
            "lambda_HML": model.params[3],
            "n_cs": int(model.nobs)
        })
    lambda_df = pd.DataFrame(lam_rows).sort_values("trade_month")

    # 对 lambda 时间序列做均值与 t 值
    def summarize_simple(series):
        s = series.dropna()
        n = len(s)
        mean = s.mean()
        std = s.std(ddof=1)
        tval = mean / (std / np.sqrt(n)) if (std > 0 and n > 1) else np.nan
        return mean, std, n, tval

    def summarize_hac(series, lags: int = 3):
        s = series.dropna().values
        n = len(s)
        if n <= 1:
            return np.nan, np.nan, n, np.nan
        # 回归: s_t = mu + e_t，取常数项的HAC标准误
        y = s
        X = np.ones((n, 1))
        model = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": lags})
        mean = model.params[0]
        se = model.bse[0]
        tval = mean / se if se > 0 else np.nan
        return mean, se, n, tval

    stats_rows = []
    if use_hac:
        for col in ["lambda_0","lambda_MKT","lambda_SMB","lambda_HML"]:
            mean, se, n, tval = summarize_hac(lambda_df[col], lags=hac_lags)
            stats_rows.append({"param": col, "mean": mean, "se_or_std": se, "n": n, "t": tval})
    else:
        for col in ["lambda_0","lambda_MKT","lambda_SMB","lambda_HML"]:
            mean, std, n, tval = summarize_simple(lambda_df[col])
            stats_rows.append({"param": col, "mean": mean, "se_or_std": std, "n": n, "t": tval})
    summary = pd.DataFrame(stats_rows)

    return step1, lambda_df, summary

############################################
# 主流程
############################################
def main(use_local_panel: bool = True,
         panel_dir: str = PANEL_DIR,
         panel_base_name: str = PANEL_BASE_NAME,
         use_hac: bool = False,
         hac_lags: int = 3):
    if not use_local_panel:
        print("1) 拉取上证50成分")
        sz50 = get_sz50_constituents()
        print(f"上证50成分股数量: {len(sz50)}")

        print("2) 计算上证50个股月度收益")
        sz50_ret = get_monthly_returns_for_stocks(sz50, START_DATE, END_DATE)
        print(f"上证50个股月收益条目: {sz50_ret.shape[0]}")

        print("3) 构造全市场 MV/PB 面板")
        mv_pb = get_universe_mv_pb(START_DATE, END_DATE)
        print(f"全市场 MV/PB 面板条目: {mv_pb.shape[0]}")

        print("4) 获取全市场月度收益并计算 FF3 因子")
        univ_ret = get_universe_monthly_returns(START_DATE, END_DATE)
        ff3 = compute_ff3_factors(univ_ret, mv_pb)
        print(f"FF3 因子月份数: {ff3.shape[0]}")

        print("5) 合并面板")
        panel = build_panel(ff3, sz50_ret)
        print(f"面板数据条目: {panel.shape[0]}")

        # 保存到本地
        save_panel(panel, out_dir=panel_dir, base_name=panel_base_name)

    else:
        print("跳过数据抓取，直接从本地加载面板...")
        panel = load_panel(in_dir=panel_dir, base_name=panel_base_name)
        print(f"本地面板数据条目: {panel.shape[0]}")

    print("6) Fama–MacBeth 两步回归")
    step1, lambda_df, summary = fama_macbeth(panel, use_hac=use_hac, hac_lags=hac_lags)

    print("\n第一步：时间序列回归（前5行）")
    print(step1.head())

    print("\n第二步：横截面回归（前5行）")
    print(lambda_df.head(50))

    print("\n第二步：因子价格统计（均值/SE或STD、样本量、t值）")
    print(summary)



# 本地读取→回归（如需 HAC，可设置 use_hac=True, hac_lags=3）
    main(use_local_panel=True, use_hac=False)
